In [267]:
import pandas as pd
import csv
import numpy as np

In [268]:
first_data = pd.read_csv('../../1995_Street_Tree_Census.csv')
second_data = pd.read_csv('../../2005_Street_Tree_Census.csv')
third_data = pd.read_csv('../../2015_Street_Tree_Census.csv')

/Users/kateharwood/opt/miniconda3/envs/climate/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (37) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [269]:
# Extract the columns we care about
first_data = first_data[["Latitude", "Longitude", "Borough", "Zip Codes", "Spc_Latin", "Diameter", "Condition"]]
second_data = second_data[["latitude", "longitude", "boroname", "zipcode", "spc_latin", "tree_dbh", "status"]]
third_data = third_data[["latitude", "longitude", "borough", "postcode", "spc_latin", "tree_dbh", "status", "health"]]

# Standardize all health statuses across the 2005 and 2015 datasets

In [270]:
# first_data
# first_data.dtypes
first_data.Condition.unique()
# first_data.Borough.unique()

array(['Unknown', 'Good', 'Excellent', 'Poor', 'Dead', 'Stump',
       'Planting Space', 'Shaft', 'Fair', 'Critical'], dtype=object)

In [271]:
# Rename "Excellent" to "Good", rename "Good" to "Fair"
second_data.status[second_data['status'] == "Good"] = "Fair"
second_data.status[second_data['status'] == "Excellent"] = "Good"
second_data = second_data.rename(columns={"status": "health"})
second_data.health.unique()



/var/folders/z7/x5j_gv2x6f92wlfl832cwwy80000gn/T/ipykernel_5205/16073105.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  second_data.status[second_data['status'] == "Excellent"] = "Good"


array(['Fair', 'Poor', 'Good', 'Dead'], dtype=object)

In [272]:
# Set all status to be "Alive" or "Dead" (remove "Stump")
third_data.health[(third_data['health'].isna()) & (third_data['status'] != "Alive")] = "Dead"
# Remove the row with status == alive and health is na
third_data.drop(third_data[third_data['health'].isna()].index, inplace = True)
# Drop extra status column
third_data.drop(columns=['status'], inplace=True)
third_data.health.unique()

# third_data.dtypes
# third_data.status.unique()
# third_data.borough.unique()
# third_data.root_stone


array(['Fair', 'Good', 'Poor', 'Dead'], dtype=object)

In [273]:
# Make categorical health data numeric
health_dict = {'Dead':0, 'Poor':1, 'Fair':2, 'Good':3}
second_data['health_status'] = second_data['health'].map(health_dict)
third_data['health_status'] = third_data["health"].map(health_dict)
second_data

,latitude,longitude,boroname,zipcode,spc_latin,tree_dbh,health,health_status
0,40.632653,-74.000245,Brooklyn,11219,PYRUS CALLERYANA,6,Fair,2
1,40.620084,-73.901453,Brooklyn,11234,PLATANUS ACERIFOLIA,6,Fair,2
2,40.617996,-73.899111,Brooklyn,11234,ACER PLATANOIDES CRIMSON KING,13,Fair,2
3,40.619694,-73.901003,Brooklyn,11234,PLATANUS ACERIFOLIA,13,Fair,2
4,40.618323,-73.899467,Brooklyn,11234,PLATANUS ACERIFOLIA,15,Fair,2
...,...,...,...,...,...,...,...,...
592367,40.586260,-74.148797,5,10314,PLATANUS ACERIFOLIA,9,Fair,2
592368,40.586090,-74.149013,5,10314,ACER SACCHARINUM,7,Fair,2
592369,40.585802,-74.149156,5,10314,ACER PLATANOIDES CRIMSON KING,5,Fair,2
592370,40.585802,-74.149156,5,10314,ACER PLATANOIDES CRIMSON KING,9,Fair,2


# Standardize feature columns across datasets

In [274]:
second_data = second_data.rename(columns={"boroname": "borough"})
third_data = third_data.rename(columns={"postcode": "zipcode"})
second_data.borough[second_data['borough'] == 5] = 'Staten Island'
# TODO have to do spc_latin too

/var/folders/z7/x5j_gv2x6f92wlfl832cwwy80000gn/T/ipykernel_5205/2318802353.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  second_data.borough[second_data['borough'] == 5] = 'Staten Island'


# Group trees into geospatial segments that match across datasets

In [275]:
# Group trees into geospatial segments by lat and long
step = 0.00000001 # This should be about 1/4-1/2 a city block 
# step = 0.001
to_bin = lambda x: np.floor(x / step) * step
second_data["latBin"] = to_bin(second_data.latitude)
second_data["lonBin"] = to_bin(second_data.longitude)

# Drop all rows where lat or long is 0
second_data.drop(second_data[second_data['latBin'] == 0].index, inplace = True)
second_data.drop(second_data[second_data['lonBin'] == 0].index, inplace = True)


In [276]:
# Set latBin and lonBin values to match across datasets
# Run this once and save output, it takes a while

third_data["latBin"] = 0
third_data["lonBin"] = 0
binValues = sorted(list(dict.fromkeys(list(zip(second_data['latBin'], second_data['lonBin'])))))

for latBinValue, lonBinValue in binValues:
    third_data.loc[third_data['latitude'] >= latBinValue, ['latBin', 'lonBin']] = latBinValue, lonBinValue


In [ ]:
# Save the processed dataframe so we don't have to compute lat and long bins every time
third_data.to_csv('../etc/data/2015_data.csv')

In [ ]:
third_data = pd.read_csv('../etc/data/2015_data.csv')
third_data.drop(columns=['lonBinValue', 'Unnamed: 0'], inplace = True) # TODO should re-run above and remove the lonBinValue part
third_data

In [ ]:
# second_data.sort_values(by='latBin', ascending=False).head(10)

In [ ]:
# third_data.sort_values(by='latBin', ascending=False)
# third_data[third_data['latBin'] > 40.912].sort_values(by='latBin', ascending=False)

,latitude,longitude,borough,zipcode,spc_latin,tree_dbh,health,health_status,latBin,lonBin
0,40.632653,-74.000245,Brooklyn,11219,PYRUS CALLERYANA,6,Fair,2,40.632653,-74.000245
1,40.620084,-73.901453,Brooklyn,11234,PLATANUS ACERIFOLIA,6,Fair,2,40.620084,-73.901453
2,40.617996,-73.899111,Brooklyn,11234,ACER PLATANOIDES CRIMSON KING,13,Fair,2,40.617996,-73.899111
4,40.618323,-73.899467,Brooklyn,11234,PLATANUS ACERIFOLIA,15,Fair,2,40.618323,-73.899467
8,40.618490,-73.915932,Brooklyn,11234,ACER PLATANOIDES CRIMSON KING,14,Good,3,40.618490,-73.915932
...,...,...,...,...,...,...,...,...,...,...
592366,40.586318,-74.148725,Staten Island,10314,PRUNUS SPECIES,9,Fair,2,40.586318,-74.148725
592368,40.586090,-74.149013,Staten Island,10314,ACER SACCHARINUM,7,Fair,2,40.586090,-74.149013
592369,40.585802,-74.149156,Staten Island,10314,ACER PLATANOIDES CRIMSON KING,5,Fair,2,40.585802,-74.149156
592370,40.585802,-74.149156,Staten Island,10314,ACER PLATANOIDES CRIMSON KING,9,Fair,2,40.585802,-74.149156


In [321]:
groups = second_data.groupby(["latBin", "lonBin"])
groups.size().sort_values().value_counts()
groups.size().sort_values()


latBin     lonBin    
40.680785  -73.769329      1
40.685319  -73.985931      1
           -73.986897      1
40.685322  -73.984845      1
           -73.972002      1
                        ... 
40.728994  -74.009677    118
40.611362  -74.187978    131
40.739677  -73.825351    132
40.625613  -74.147311    263
40.693685  -73.983339    342
Length: 268077, dtype: int64

In [ ]:
# Get third data into same lat and long bins as second data
